# 도로점용허가내용 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "도로점용허가내용 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/도로점용허가내용 목록/selectIoPmPermitList.csv'

In [2]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [3]:
JSONKEY = "items"
DUMMY = 0

In [4]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

In [5]:
resultDfMerged = pd.DataFrame()

In [6]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
endYr

'2022'

In [7]:
# 기준년도 산정
# 2001년 부터 데이터 존재
startYr = "2001"
dateRange = list(map(int,pd.period_range(startYr,endYr,freq='Y').strftime('%Y').tolist()))
# dateRange

In [11]:
# 도로점용허가 목록 수
g_totalCount = 0

## 도로점용허가내용 목록 전용 스크랩 함수

In [12]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [16]:
numOfRows = 1000
resultDf = pd.DataFrame()
for i  in dateRange:
    pageNo = 1
    print(i)
    global g_totalCount
    g_totalCount = 0
    while True:
        BASEPARAM={"serviceKey":APIKEY,"searchEdBgDt": str(i)+"0101", "searchEdEdDt": str(i)+"1231", "sortField": "edDt","sortOrder": "ASC", "pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
        #print("BASEPARAM ",BASEPARAM)
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        if resultDf.empty: # 정상 데이터가 없는 경우
            print("No Data")
            break
        else:
            resultDfMerged = resultDfMerged.append(resultDf)
            g_totalCount -= numOfRows
            print("g_totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
            if(g_totalCount <= 0):
                break
            pageNo += 1

2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20010101', 'searchEdEdDt': '20011231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 12
0 no pageNo
g_totalCount :  -988  numOfRows :  1000
2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20020101', 'searchEdEdDt': '20021231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6
0 no pageNo
g_totalCount :  -994  numOfRows :  1000
2003
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20030101', 'searchEdEdDt': '20031231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 4
0 no pageNo
g_totalCount :  -996  numOfRows :  1000
2004
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': 

>totalCount 3719
0 no pageNo
g_totalCount :  -281  numOfRows :  1000
2013
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20130101', 'searchEdEdDt': '20131231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 4525
0 no pageNo
g_totalCount :  3525  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20130101', 'searchEdEdDt': '20131231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 2, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 4525
0 no pageNo
g_totalCount :  2525  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20130101', 'searchEdEdDt': '20131231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 3, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 4525
0 no pageNo
g_totalCount :  1525  numOfRows :  1000
BASEPARAM  {'service

>totalCount 6858
0 no pageNo
g_totalCount :  5858  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20170101', 'searchEdEdDt': '20171231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 2, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6858
0 no pageNo
g_totalCount :  4858  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20170101', 'searchEdEdDt': '20171231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 3, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6858
0 no pageNo
g_totalCount :  3858  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20170101', 'searchEdEdDt': '20171231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 4, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6858
0 no pageNo
g_totalCount :  2858  numOfRows :  1000
BASEPARAM  {'serviceKey':

>totalCount 6717
0 no pageNo
g_totalCount :  5717  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20210101', 'searchEdEdDt': '20211231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 2, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6717
0 no pageNo
g_totalCount :  4717  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20210101', 'searchEdEdDt': '20211231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 3, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6717
0 no pageNo
g_totalCount :  3717  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchEdBgDt': '20210101', 'searchEdEdDt': '20211231', 'sortField': 'edDt', 'sortOrder': 'ASC', 'pageNo': 4, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6717
0 no pageNo
g_totalCount :  2717  numOfRows :  1000
BASEPARAM  {'serviceKey':

In [17]:
resultDfMerged

,rqsno,prmcd,prmnm,orcd,ornm,lgnid,eddt,edtycd,edrsn,feects,perno,pmsnrectmtdcd,rlappinm,onlpmsnno,lstupdde,masterlinkcnt
0,A010200112M002,A010,도로점용 허가,A4020,전주국토관리사무소,None,2001-11-20,1,허가,None,전주제2001-81호,9,정명재,125581,20080903,0
1,A010200111M008,A010,도로점용 허가,A4020,전주국토관리사무소,None,2001-11-20,1,허가,None,전주 제2001-80호,9,이수미,114983,20080903,0
2,A010200111M015,A010,도로점용 허가,A4020,전주국토관리사무소,None,2001-11-27,1,허가,None,전주제2001-83호,9,한국전력공사전북지사장,115479,20080903,0
3,A010200111M016,A010,도로점용 허가,A4020,전주국토관리사무소,None,2001-11-28,1,허가,None,전주제2001-84호,9,김기춘,115177,20080903,0
4,A010200111M018,A010,도로점용 허가,A4020,전주국토관리사무소,None,2001-12-01,1,허가,None,전주제2001-85호,9,정읍시장,115684,20080903,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,A012202203M058,A012,도로점용허가 변경,A5020,진주국토관리사무소,None,2022-03-31,1,변경 허가,None,진주-2021-024,None,한국전력공사 산청지사장,220357227,20220331,0
404,A010202203M029,A010,도로점용 허가,A5040,영주국토관리사무소,None,2022-03-31,1,귀 사에서 접수하신 도로점용(굴착)허가 신청에 대하여는 신청한 내용대로 허가합니다....,None,영22-010,None,한전상주지점장,220356549,20220331,0
405,A053202203M031,A053,도로와 다른 시설의 연결허가 변경,A5040,영주국토관리사무소,None,2022-03-31,1,귀 하께서 접수하신 도로점용 변경허가 신청에 대하여 신청한 내용대로 변경허가 합니다...,None,영21-008,None,류형,220394859,20220331,0
406,A053202203M032,A053,도로와 다른 시설의 연결허가 변경,A5040,영주국토관리사무소,None,2022-03-31,1,귀 하께서 접수하신 도로점용 변경허가 신청에 대하여는 신청한 내용대로 변경허가합니다...,None,영19-042,None,임영식,220342081,20220331,0


In [18]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

도로점용허가내용 목록 save compled


In [19]:
resultDfMerged.shape

(88244, 16)